In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta

## Define dataset and variables

In [2]:
case = "fanv3_debugging"
path = "/glade/scratch/jinmuluo/archive/fanv3only_clmnox_correctedsgd/lnd/hist/" 
start_date = date(2000, 1, 1)
end_date = date(2000, 12, 1)
delta = end_date - start_date

In [3]:
fieldsh0 = ["FERT_NO3_TO_SOIL", "MANURE_NO3_TO_SOIL", "MANURE_NITRATE_RUNOFF", 'FERT_NITRATE_RUNOFF', 
            "FERT_NITRATE_TO_SOIL", "MANURE_NITRATE_TO_SOIL", 'area']
fieldsh2 = ["N2O_DENITRIFY_TOTAL", "N2O_NITRIFY_TOTAL", "NOx_DENITRIFY_TOTAL", "NOx_NITRIFY_TOTAL", 
            "N2_TOTAL", "area"]
fieldsh3 = ["NITRATE_G1", "NITRATE_G2", "NITRATE_G3", "NITRATE_S0", "NITRATE_S1", "NITRATE_S2",
            "NITRATE_S3", "NITRATE_F1", "NITRATE_F2",  "NITRATE_F3",  "NITRATE_F4", 'area']

## (1) Nitrification flux from fanv2

In [4]:
fanCLM = []
for i in range(delta.days//30 + 1):
    month = start_date + relativedelta(months=i)
    month = month.strftime('%Y-%m')
    fanCLM.append(path + case + ".clm2." + "h0" + "." + month +".nc")
# print(fanCLM)

In [5]:
def preprocess(ds, fields = fieldsh0):
    return(ds[fields])

dsCLM = xr.open_mfdataset(fanCLM, decode_times=True, preprocess=preprocess)

In [6]:
ds = (dsCLM["FERT_NO3_TO_SOIL"] + dsCLM["MANURE_NO3_TO_SOIL"]) * dsCLM['area']*1e6
ds = ds.sum(dim=['lat', 'lon'])

fanv2_flux = 0.0

# unit transfer form gN/s to Tg/year
for i in range(len(ds.time)):
    t= start_date + relativedelta(months=i)
    t2 = start_date + relativedelta(months=i+1)
    day = (t2-t).days
    fanv2_flux = fanv2_flux + ds[i].values * day * 24 * 3600
    
fanv2_flux = fanv2_flux * 1e-12

print(fanv2_flux, "Tg")

18.8686563102 Tg


## (2) NOx, N2O and N2 emission 

In [7]:
fanCLM = []
for i in range(delta.days//30 + 1):
    month = start_date + relativedelta(months=i)
    month = month.strftime('%Y-%m')
    fanCLM.append(path + case + ".clm2." + "h2" + "." + month +".nc")

In [8]:
def preprocess(ds, fields = fieldsh2):
    return(ds[fields])

dsCLM = xr.open_mfdataset(fanCLM, decode_times=True, preprocess=preprocess)
# dsCLM

In [9]:
ds_n2o = (dsCLM["N2O_DENITRIFY_TOTAL"] + dsCLM["N2O_NITRIFY_TOTAL"]) * dsCLM['area']*1e6
ds_nox = (dsCLM["NOx_DENITRIFY_TOTAL"] + dsCLM["NOx_NITRIFY_TOTAL"]) * dsCLM['area']*1e6
ds_n2 = dsCLM["N2_TOTAL"] * dsCLM['area']*1e6


ds_n2o = ds_n2o.sum(dim=['lat', 'lon'])
ds_nox = ds_nox.sum(dim=['lat', 'lon'])
ds_n2 = ds_n2.sum(dim=['lat', 'lon'])

n2o_total = 0.0
nox_total = 0.0
n2_total = 0.0

# unit transfer form gN/s to Tg/year
for i in range(len(ds.time)):
    t= start_date + relativedelta(months=i)
    t2 = start_date + relativedelta(months=i+1)
    day = (t2-t).days
    n2o_total = n2o_total + ds_n2o[i].values * day * 24 * 3600
    nox_total = nox_total + ds_nox[i].values * day * 24 * 3600
    n2_total = n2_total + ds_n2[i].values * day * 24 * 3600
    
n2o_total = n2o_total * 1e-12
nox_total = nox_total * 1e-12
n2_total = n2_total * 1e-12

print('N2O:', n2o_total, "Tg")
print('NOx:', nox_total, "Tg")
print('N2', n2_total, "Tg")

N2O: 0.341449600584375 Tg
NOx: 1.4980718491875 Tg
N2 0.008064906746044922 Tg


## (3) Diffusion and Runoff

In [10]:
fanCLM = []
for i in range(delta.days//30 + 1):
    month = start_date + relativedelta(months=i)
    month = month.strftime('%Y-%m')
    fanCLM.append(path + case + ".clm2." + "h0" + "." + month +".nc")

In [11]:
def preprocess(ds, fields = fieldsh0):
    return(ds[fields])

dsCLM = xr.open_mfdataset(fanCLM, decode_times=True, preprocess=preprocess)
#dsCLM

In [12]:
ds_soil = (dsCLM["FERT_NITRATE_TO_SOIL"] + dsCLM["MANURE_NITRATE_TO_SOIL"]) * dsCLM['area']*1e6
ds_runoff = (dsCLM["MANURE_NITRATE_RUNOFF"] + dsCLM["FERT_NITRATE_RUNOFF"]) * dsCLM['area']*1e6
ds_soil = ds_soil.sum(dim=['lat', 'lon'])
ds_runoff = ds_runoff.sum(dim=['lat', 'lon'])

soil = 0.0
runoff = 0.0

# unit transfer form gN/s to Tg/year
for i in range(len(ds.time)):
    t= start_date + relativedelta(months=i)
    t2 = start_date + relativedelta(months=i+1)
    day = (t2-t).days
    soil = soil + ds_soil[i].values * day * 24 * 3600
    runoff = runoff + ds_runoff[i].values * day * 24 * 3600
    
soil = soil * 1e-12
runoff = runoff * 1e-12

print("diffusion to soil", soil, "Tg")
print("surface runoff", runoff, "Tg")

diffusion to soil 14.4116501319 Tg
surface runoff 1.6252622730843749 Tg


## (4) Nitrite pools

In [15]:
fanCLM = []
for i in range(delta.days//30 + 2):
    if start_date.month != 1:
        date_term = date(start_date.year, start_date.month - 1, start_date.day)
    else:
        date_term = start_date
    month = date_term + relativedelta(months=i)
    month = month.strftime('%Y-%m')
    fanCLM.append(path + case + ".clm2." + "h3" + "." + month +".nc")

In [16]:
dsCLM_end = xr.open_dataset(fanCLM[-1])
dsCLM_begin = xr.open_dataset(fanCLM[0])
# dsCLM_begin

In [17]:
pools_end = 0.0
pools_begin = 0.0

for pool in fieldsh3:
    if pool != 'area':
        pools_end = pools_end + (dsCLM_end[pool] * dsCLM_end["area"] * 1e6).sum(dim=['lat', 'lon']).values
        pools_begin = pools_begin + (dsCLM_begin[pool] * dsCLM_begin["area"] * 1e6).sum(dim=['lat', 'lon']).values

# g to Tg
pools_end = pools_end * 1e-12
pools_begin = pools_begin * 1e-12
nitrate = pools_end - pools_begin
print("nitrate pools at begin:", pools_begin)
print("nitrate pools in the end:", pools_end)
print("nitrate pools: ", nitrate, "Tg")

nitrate pools at begin: [0.324297]
nitrate pools in the end: [0.7558494]
nitrate pools:  [0.4315524] Tg


## N Balance

In [18]:
d = fanv2_flux - n2o_total - nox_total - n2_total - runoff - soil - nitrate
print(d)

[0.55260515]
